# Foward/Backward Computation

In [69]:
import chainer
import chainer.functions as F
import chainer.links as L
import numpy as np
from chainer import (Chain, ChainList, Function, Variable, datasets,
                     gradient_check, iterators, link, optimizers, report,
                     serializers, training, utils)
from chainer.training import extensions

In [2]:
x_data = np.array([5], dtype=np.float32)
x = Variable(x_data)  # trainableな変数となる

In [3]:
x, x.data  # can get value from data attribute then it's array

(variable([5.]), array([5.], dtype=float32))

In [4]:
y = x**2 -2 * x +1

In [5]:
y.data

array([16.], dtype=float32)

In [6]:
y.backward()  # backwordを行うと
x.grad  # それぞれの変数においての勾配が計算される．

array([8.], dtype=float32)

In [7]:
z = 2*x
y = x**2 - z +1
y.backward(retain_grad=True)
z.grad  # 中間の変数の勾配はメモリ効率のため，retain_grad=Trueとしないと解放される．

array([-1.], dtype=float32)

In [8]:
y.backward()  # そうしないと勾配を保持しない

In [9]:
z.grad is None

True

In [10]:
x.grad  # 最初の変数は勾配が常にある．

array([24.], dtype=float32)

In [11]:
x = Variable(np.array([[1, 2 ,3], [4, 5, 6]], dtype=np.float32))
y = x**2 -2*x +1
# 多次元配列を入力とするときは初期誤差を明示的に書かなければならない
y.grad = np.ones((2, 3), dtype=np.float32)
y.backward()
x.grad

array([[ 0.,  2.,  4.],
       [ 6.,  8., 10.]], dtype=float32)

In [12]:
x = Variable(np.array([[1, 2 ,3], [4, 5, 6]], dtype=np.float32))
y = x**2 -2*x +1
# y.grad = np.ones((2, 3), dtype=np.float32)
y.backward()
x.grad is None

True

# Links

In [13]:
# accept input value whose shape is (N, 3) and this functions output shape is (N,2)
f = L.Linear(3, 2)

In [14]:
f.W  # instance of Variable

variable W([[-0.06382784, -0.6825712 ,  0.7025081 ],
            [ 0.51501   ,  0.5847196 , -0.8228216 ]])

In [15]:
f.W.data, f.b.data  # W,b はインスタンス化した際に初期化される．よって何度このセルを実行しても値は同じ

(array([[-0.06382784, -0.6825712 ,  0.7025081 ],
        [ 0.51501   ,  0.5847196 , -0.8228216 ]], dtype=float32),
 array([0., 0.], dtype=float32))

In [16]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]],
                      dtype=np.float32))  # input
y = f(x)  # fully-connected
y.data

array([[ 0.6785542 , -0.78401566],
       [ 0.5468817 ,  0.04670811]], dtype=float32)

In [17]:
f = L.Linear(2)

In [18]:
f.W, f.b  # 入力次元を指定してないので, W はデータが流れるまでは初期化されない

(variable W(None), variable b([0., 0.]))

In [19]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]],
                      dtype=np.float32))  # input
y = f(x)  # fully-connected
print(f.W, f.b)  # 推論したら値が初期化されている．
y.data

variable W([[-0.14056611  0.1022629  -0.38314268]
            [ 0.4181937   0.02477389  0.14322315]]) variable b([0. 0.])


array([[-1.0854683,  0.8974109],
       [-2.3498058,  2.655983 ]], dtype=float32)

In [20]:
f.cleargrads()

In [21]:
y.grad = np.ones((2, 2), dtype=np.float32)
y.backward()
f.W.grad, f.b.grad

(array([[5., 7., 9.],
        [5., 7., 9.]], dtype=float32), array([2., 2.], dtype=float32))

In [22]:
y.backward()

In [23]:
f.W.grad, f.b.grad  # cleargradsをしないと蓄積される

(array([[10., 14., 18.],
        [10., 14., 18.]], dtype=float32), array([4., 4.], dtype=float32))

In [24]:
f.cleargrads()
y.grad = np.ones((2, 2), dtype=np.float32)
y.backward()
f.W.grad, f.b.grad

(array([[5., 7., 9.],
        [5., 7., 9.]], dtype=float32), array([2., 2.], dtype=float32))

# Write a model as a chain

In [25]:
l1 = L.Linear(4, 3)
l2 = L.Linear(3, 2)
def my_forward(x):
    h = l1(x)
    return l2(h)

In [26]:
class MyProc(object):
    def __init__(self):
        self.l1 = L.Linear(4, 3)
        self.l2 = L.Linear(3, 2)
        
    def forward(self, x):
        h = self.l1(x)
        return self.l2(h)

In [27]:
class MyChain(Chain):
    def __init__(self):
        # MyChainクラスのスーパークラスの初期化メソッドを呼ぶ．単にsuper().__init__でもOK
        super(MyChain, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(4, 3)
            self.l2 = L.Linear(3, 2)
        
    def __call__(self, x):
        h = self.l1(x)
        return self.l2(h)

In [28]:
class MyChain2(ChainList):
    def __init__(self):
        # MyChain2クラスのスーパークラスの初期化メソッドを呼ぶ．単にsuper().__init__でもOK
        super(MyChain2, self).__init__(
            L.Linear(4, 3),
            L.Linear(3, 2),
        )

        
    def __call__(self, x):
        h = self[0](x)
        return self[1](h)

# Optimizer

In [29]:
model = MyChain()
optimizer = optimizers.SGD(lr=0.01)
optimizer.setup(model)

In [30]:
model, optimizer

(<__main__.MyChain at 0x7fd8053a7e80>,
 <chainer.optimizers.sgd.SGD at 0x7fd8053a7dd8>)

In [31]:
# あらかじめadd_hookメソッドを呼ぶことで正則化などができる
optimizer.add_hook(chainer.optimizer.WeightDecay(0.0005))

In [32]:
x = np.random.uniform(-1, 1, (2, 4)).astype('f')
x

array([[ 0.31473038,  0.52423704, -0.5810844 ,  0.61855686],
       [-0.76510185, -0.01017925, -0.22086169,  0.06795558]],
      dtype=float32)

In [33]:
model.cleargrads()

# chainerを使って定義した機械学習ネットワークにデータを投げる時には型をVariableにしなければいけません。
# その時、引数に与えるデータは常にnp.arrayを用いる必要があります。
loss = model(chainer.Variable(x))  # define loss function
loss

variable([[-0.32220206, -0.44709098],
          [ 0.8375386 , -0.04884314]])

In [34]:
loss = F.sum(loss)
loss

variable(0.01940239)

In [35]:
loss.backward()  # compute gradients here
optimizer.update()

In [36]:
model.l1.W.grad

array([[-0.70729434,  0.8072384 , -1.2585516 ,  1.0778688 ],
       [-0.19065623,  0.21674514, -0.33819425,  0.2896081 ],
       [-0.18735579,  0.2138383 , -0.33366752,  0.28530058]],
      dtype=float32)

In [37]:
model.cleargrads()  # 勾配消去
loss.backward()  # compute gradient
optimizer.update()  # 最適化
print(model.l1.W.data)  # 勾配を消して,backwordしてupdateすれば，変数が最適化されていく
model.l1.W.grad

[[-0.44885355  0.3657488   0.9597713   0.129384  ]
 [-1.505267   -0.02484447 -0.09147666  0.26664013]
 [-0.09014332  0.08053309 -0.271929   -0.44358745]]


array([[-0.7094576 ,  0.8097076 , -1.2624037 ,  1.0811665 ],
       [-0.1822358 ,  0.207134  , -0.32320055,  0.27677262],
       [-0.18727562,  0.21374677, -0.33352473,  0.28517836]],
      dtype=float32)

In [38]:
model2 = MyChain()
optimizer = optimizers.SGD(lr=0.01)
optimizer.setup(model2)

In [39]:
np.random.seed(1)
def lossfun(arg1, arg2):
    # calculate loss
    loss = F.sum(model2(arg1 - arg2))
    return loss
arg1 = np.random.uniform(-1, 1, (100, 4)).astype('f')  # 入力データ1
arg2 = np.random.uniform(-1, 1, (100, 4)).astype('f')  # 入力データ2
# lossfunc以外の引数はlossfuncの引数である必要がある. この方法での最適化だとcleargradsは必要ない
optimizer.update(lossfun, chainer.Variable(arg1), chainer.Variable(arg2)) 
print(model2.l1.W.data)
model.l1.W.grad

[[ 0.5247217   0.26374596  0.04687392 -0.10371631]
 [ 0.33678135  0.39854538 -0.55439186 -0.9291288 ]
 [ 0.06033649 -0.07023801 -0.6669371   0.05834466]]


array([[-0.7094576 ,  0.8097076 , -1.2624037 ,  1.0811665 ],
       [-0.1822358 ,  0.207134  , -0.32320055,  0.27677262],
       [-0.18727562,  0.21374677, -0.33352473,  0.28517836]],
      dtype=float32)

`F.mean_squared_error`をつかってみる．

In [57]:
model3 = MyChain()
optimizer = optimizers.SGD(lr=0.01)
optimizer.setup(model3)
np.random.seed(1)

def lossfun(arg1, arg2):
    # calculate loss
    loss = F.mean_squared_error(model3(arg1), arg2)
    return loss

arg1 = np.random.uniform(-1, 1, (100, 4)).astype('f')  # 入力データ1
arg2 = np.random.uniform(-1, 1, (100, 2)).astype('f')  # 入力データ2
# lossfunc以外の引数はlossfuncの引数である必要がある. この方法での最適化だとcleargradsは必要ない
optimizer.update(lossfun, chainer.Variable(arg1), chainer.Variable(arg2)) 
print(model3.l1.W.data)
model3.l1.W.grad

[[ 0.52021104  0.30367592 -0.03390292 -0.05569299]
 [ 0.22469133  0.882432    0.43537852 -0.2549535 ]
 [ 0.38438904 -0.06087445 -0.09872811  0.9290657 ]]


array([[ 0.22861126,  0.05814299, -0.07614249,  0.14969613],
       [ 0.0386434 ,  0.02355422,  0.01064028,  0.07249323],
       [ 0.43205643,  0.14888667, -0.07709835,  0.41699764]],
      dtype=float32)

# Trainer

In [70]:
serializers.save_npz("my.model", model)  # serialize Chain object

In [73]:
# モデルしたいを保存するわけではないので，ロードしたパラメータ値を入れるモデルは予め作っておかないといけない．
# 引数に渡すモデルはsaveした時と同じ構成である必要がある
# Since we do not preserve the model itself, we need to create a model that contains loaded parameter values in advance.
serializers.load_npz("my.model", model)
model

In [76]:
# serialize state of optimizer
serializers.save_npz("my.state", optimizer)
serializers.load_npz("my.state", optimizer)

# Example: Multi-layer Perceptron on MNIST